# Web search using Brave Search Engine

This notebook provides an introduction to connecting Claude to web search data by performing retrieval-augmented generation (RAG) with the help of the Brave search engine. Specifically, this notebook will walk you through:
1. Getting search results from the Brave API
2. Using Claude to answer questions with information from the real-time search data

## Steps
1. Setup
2. Searching with the Brave search API
3. Inject search results into prompt
4. Answer the user's question with Claude


## Setup
First, let's install the necessary libraries and set the API keys we will need to use in this notebook. We will need to get our [Claude API key](https://docs.anthropic.com/claude/reference/getting-started-with-the-api) and our free [Brave API key](https://brave.com/search/api/). Make sure to select the "Data for AI" plan from Brave when choosing a subscription. 

In [ ]:
%pip install anthropic requests beautifulsoup4

In [74]:
# Insert your API keys here
ANTHROPIC_API_KEY="<your_anthropic_api_key>"
BRAVE_API_KEY="<your_brave_api_key>"

## Searching with Brave search API
Now that our environment is setup, let's test the Brave search API. Here's a user's question we want to find an answer to:

In [75]:
USER_QUESTION="What show won the Outstanding Drama award at the 2024 Emmys?"

Now, we could directly feed this question to the Brave search API or we could use Claude to generate a list of diverse search queries to try based on the question to make our search more exhaustive. For the purposes of this notebook, we will do the latter.

In [76]:
GENERATE_QUERIES=f"""\n\nHuman: You are an expert at generating search queries for the Brave search engine.
Generate three search queries that are relevant to this question. Output only valid JSON.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}\n\nAssistant: {{"""

In [77]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def get_completion(prompt: str):
    message = client.completions.create(
        model='claude-2.1',
        max_tokens_to_sample=1024,
        temperature=.5,
        prompt=prompt
    )
    return message.completion
queries_json = "{" + get_completion(GENERATE_QUERIES)
print(queries_json)


{
  "queries": [
    "2024 emmy awards outstanding drama winner",
    "2024 emmy awards outstanding drama series winner",
    "who won outstanding drama emmy in 2024"
  ]
}


Now let's get the top three search results for each one of the search queries Claude generated. Notice that we make a set of the urls to dedupe our results since some of the results are the same across different queries.

In [ ]:
import requests
from time import sleep
import json

def get_search_results(search_query : str):
    headers = {"Accept": "application/json", "X-Subscription-Token": BRAVE_API_KEY}
    response = requests.get(
        "https://api.search.brave.com/res/v1/web/search",
        params={"q": search_query,
                "count": 3 # Max number of results to return
                },
        headers=headers,
        timeout=60
    )
    if not response.ok:
        raise Exception(f"HTTP error {response.status_code}")
    sleep(1) # avoid Brave rate limit
    return response.json().get("web", {}).get("results")
queries = json.loads(queries_json)["queries"]

urls_seen = set()
web_search_results = []
for query in queries:
    search_results = get_search_results(query)
    for result in search_results:
        url = result.get("url")
        if not url or url in urls_seen:
            continue
        
        urls_seen.add(url)
        web_search_results.append(result)
        
print(len(web_search_results))

Let's take a closer look at the results we recieved.

In [79]:
for i, item in enumerate(web_search_results):
    print(f"Search result {i+1}:")
    print(item.get("title"))
    print(item.get("url"))

Search result 1:
Who won Emmy Awards for 2024? See the full winners list here - ...
https://www.cbsnews.com/news/emmys-winners-2024-list/
Search result 2:
2024 Primetime Emmy Awards (Full Winners List) – Billboard
https://www.billboard.com/music/awards/2024-primetime-emmy-awards-winners-list-1235580458/
Search result 3:
2023–2024 Emmy Awards Winners: The Complete List
https://www.vulture.com/2024/01/emmy-awards-2023-2024-winners.html
Search result 4:
Here Are the Winners of the 2024 Emmy Awards
https://www.tvguide.com/news/2024-emmy-winners-the-complete-list/
Search result 5:
Emmys 2024: See All the Winners Here | Vanity Fair
https://www.vanityfair.com/hollywood/awards-insider-emmy-winners-list-2024
Search result 6:
Every Winner at the 2024 Emmys, From Ayo Edebiri to Quinta Brunson
https://www.teenvogue.com/story/emmys-2023-winners-see-the-full-list-here


Great! It looks like we are getting back the web page title, url, and more metadata. To see a full description of the response object, check out the Brave search API [docs page](https://api.search.brave.com/app/documentation/web-search/get-started).

Unfortunately, we don't get back the actual web page content through the Brave search API so we need to do a little digging. Let's create a function that fetches the page content given a url.

In [80]:
from bs4 import BeautifulSoup

def get_page_content(url : str) -> str:
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(strip=True, separator='\n')

Now, let's use this on the results we fetched and format them in a style we will want to use in our prompt later on.

In [81]:
formatted_search_results = "\n".join(
        [
            f'<item index="{i+1}">\n<source>{result.get("url")}</source>\n<page_content>\n{get_page_content(result.get("url"))}\n</page_content>\n</item>'
            for i, result in enumerate(web_search_results)
        ]
    )
print(formatted_search_results)

<item index="1">
<source>https://www.cbsnews.com/news/emmys-winners-2024-list/</source>
<page_content>
Who won Emmy Awards for 2024? See the full winners list here - CBS News
Latest
U.S.
World
Politics
Entertainment
HealthWatch
MoneyWatch
Crime
Sports
Local News
Baltimore
Bay Area
Boston
Chicago
Colorado
Detroit
Los Angeles
Miami
Minnesota
New York
Philadelphia
Pittsburgh
Sacramento
Texas
Live
CBS News Streaming
Baltimore
Bay Area
Boston
Chicago
Colorado
Detroit
Los Angeles
Miami
Minnesota
New York
Philadelphia
Pittsburgh
Sacramento
Texas
Shows
48 Hours
60 Minutes
America Decides
CBS Evening News
CBS Mornings
CBS News Eye on America
CBS News Mornings
CBS Reports
CBS Saturday Morning
The Dish
Face the Nation
Here Comes the Sun
Person to Person
Prime Time
Sunday Morning
The Takeout
The Uplift
Weekender
Photos
Podcasts
In Depth
Newsletters
Mobile
CBS News Team
Executive Team
CBS Store
Paramount+
Join Our Talent Community
RSS
A Moment With...
Davos 2023
Innovators & Disruptors
U.S.
World
P

## Inject search results into prompt
Now that we have our search results nicely formatted, let's insert them into the prompt we will send to Claude.

In [97]:
ANSWER_QUESTION = f"""\n\nHuman: I have provided you with the following search results:
{formatted_search_results}

Please answer the user's question using only information from the search results. Reference the relevant search result urls within your answer as links. Keep your answer concise.

User's question: {USER_QUESTION} \n\nAssistant:
"""
print(ANSWER_QUESTION)



Human: I have provided you with the following search results:
<item index="1">
<source>https://www.cbsnews.com/news/emmys-winners-2024-list/</source>
<page_content>
Who won Emmy Awards for 2024? See the full winners list here - CBS News
Latest
U.S.
World
Politics
Entertainment
HealthWatch
MoneyWatch
Crime
Sports
Local News
Baltimore
Bay Area
Boston
Chicago
Colorado
Detroit
Los Angeles
Miami
Minnesota
New York
Philadelphia
Pittsburgh
Sacramento
Texas
Live
CBS News Streaming
Baltimore
Bay Area
Boston
Chicago
Colorado
Detroit
Los Angeles
Miami
Minnesota
New York
Philadelphia
Pittsburgh
Sacramento
Texas
Shows
48 Hours
60 Minutes
America Decides
CBS Evening News
CBS Mornings
CBS News Eye on America
CBS News Mornings
CBS Reports
CBS Saturday Morning
The Dish
Face the Nation
Here Comes the Sun
Person to Person
Prime Time
Sunday Morning
The Takeout
The Uplift
Weekender
Photos
Podcasts
In Depth
Newsletters
Mobile
CBS News Team
Executive Team
CBS Store
Paramount+
Join Our Talent Community
RSS


## Answer question using search results
Finally, now that we have our prompt constructed, we can ask Claude to answer the user's original question.

In [98]:
print(get_completion(ANSWER_QUESTION))

 Based on the search results, the show that won Outstanding Drama Series at the 2024 Emmys was <a href="https://www.cbsnews.com/news/emmys-winners-2024-list/">Succession</a>. The HBO drama series took home the top drama prize for the third time after its fourth and final season.
